In [4]:
import pandas as pd
from collections import defaultdict
import dvc.api
from tqdm import tqdm

In [5]:
df_original = pd.read_csv(dvc.api.get_url('modelling/notebooks/query_attr_extract_appen_label/appen_query_attribution_batch1.csv', 
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'))

In [6]:
df = pd.read_excel('appen/output_batch/Query Attribution_02.03.23.xlsx')

In [7]:
df.head(2)

,Label Ordering,Sample Method,Query,Translated Query,L2 Category,Query Category,Rater Answer
0,18,head,fish holder tool,fish holder tool,Sports > Fishing,Sports > Fishing > Fishing Tools,['Fishing > Fishing Tools > fish holder']
1,225,head,molde gesso 3d silicone,3D silicone chalking mold,"Home & Garden > Arts, Crafts & Sewing","Home & Garden > Arts, Crafts & Sewing > Button...","['Arts, Crafts & Sewing > Pottery & Ceramics >..."


In [8]:
df_original.head(2)

,query,sample_method,label_ordering,top_query_classification_taxonomy
0,dog cat eater,uniform,0,Home & Garden > Pet Products > Cat Supplies > ...
1,fish holder tool,head,18,Sports > Fishing > Fishing Tools


In [9]:
df = df.merge(df_original[['label_ordering', 'query', 'top_query_classification_taxonomy']].rename(columns={
    'query': 'query_original',
    'label_ordering': 'Label Ordering'
}), on='Label Ordering', how='inner')

In [10]:
(df['query_original'] != df['Query']).mean()

0.0646146459864236

In [17]:
df[df['query_original'] != df['Query']][['Label Ordering', 'Query', 'query_original']].sample(2).to_dict('records')

[{'Label Ordering': 1457176,
  'Query': 'Ã¨â€šÂ©Ã£Â\x81â€œÃ£â€šÅ\xa0 Ã£Æ’Â\x8dÃ£Æ’Æ’Ã£â€šÂ¯Ã£Æ’Â¬Ã£â€šÂ¹ Ã£â€šÂ¹Ã£Æ’Â\x9dÃ£Æ’Â¼Ã£Æ’â€ž',
  'query_original': '肩こり ネックレス スポーツ'},
 {'Label Ordering': 347362,
  'Query': 'limpador bucal aquÃƒÂ¡tico',
  'query_original': 'limpador bucal aquático'}]

In [12]:
(df['Query Category'] != df['top_query_classification_taxonomy']).mean()

0.0

In [15]:
error_c = 0
errors = []
for i in tqdm(df[['Query', 'Rater Answer', 'Query Category']].to_dict('records')):
    for j in eval(i['Rater Answer']):
        if not j.startswith(i['Query Category']):
            error_c += 1
            errors.append(i)
            break
error_c / len(df)

100%|██████████| 74983/74983 [00:00<00:00, 76621.66it/s]


1.0

In [16]:
# query errors
errors[:2]

[{'Query': 'fish holder tool',
  'Rater Answer': "['Fishing > Fishing Tools > fish holder']",
  'Query Category': 'Sports > Fishing > Fishing Tools'},
 {'Query': 'molde gesso 3d silicone',
  'Rater Answer': "['Arts, Crafts & Sewing > Pottery & Ceramics > Clay Molds > Materials > Silicone', 'Arts, Crafts & Sewing > Pottery & Ceramics > Clay Molds > Pattern > 3D']",
  'Query Category': 'Home & Garden > Arts, Crafts & Sewing > Button & Badge Making > Maker Molds'}]

In [6]:
df_meta = pd.read_csv('../attribute_extraction_metadata_template/attribute_definition_top25L2_filtered_augmented_01162023.csv')

In [24]:
# path -> attribute_field -> {"category_attributevalue": set(), "augment": False, "usage": ""}
meta_dict = defaultdict(lambda: defaultdict(lambda: {"category_attributevalue": set()})) 
for i in df_meta.to_dict('records'):
    meta_dict[i['category']][i['attribute_field']]["category_attributevalue"].update(eval(i["category_attributevalue"]))
    meta_dict[i['category']][i['attribute_field']]['augment'] = i["augment"]
    meta_dict[i['category']][i['attribute_field']]['usage'] = i["usage"]

In [ ]:
meta_dict['Home & Garden > Arts, Crafts & Sewing > Button & Badge Making > Maker Molds']['Materials']